In [ ]:
import gc
gc.collect()

In [1]:
from gill import models
from gill import utils
from torchmetrics import BLEUScore

import IPython.display as ipd
import pandas as pd
import librosa
import json
import os

In [2]:
!ls ../../../../../mnt/media/wiseyak/reasoning-datasets/AudioCaps/results

125m-25epochs-default-18_ckpt_best.json  1_ckpt_best.json  7_ckpt.json
125m-25epochs-default-3_ckpt_best.json	 2_ckpt_best.json  9_ckpt.json
125m-25epochs-default-4_ckpt.json	 5_ckpt.json


In [3]:
base_test_dir = "../../../../../mnt/media/wiseyak/reasoning-datasets/AudioCaps/"
test_df = pd.read_csv(os.path.join(base_test_dir, "test-10s-downloaded.csv"))
bleu_scorers = [BLEUScore(n_gram=i) for i in [1, 2, 3, 4]]

/mnt/media/wiseyak/reasoning/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


In [5]:
train_df = pd.read_csv(os.path.join(base_test_dir, "train-10s-downloaded.csv"))
len(train_df["youtube_id"].unique()), len(train_df)

(31040, 31040)

In [6]:
len(test_df["youtube_id"].unique()), len(test_df)

(622, 3110)

In [3]:
def join_result_with_test(test_df,ckpt_name):
    with open(os.path.join(base_test_dir,ckpt_name)) as f:
        results = json.load(f)
    generated_captions = []
    for index,row in test_df.iterrows():
        audiocap_id = row["audiocap_id"]
        generated_captions.append(results[str(audiocap_id)])  
    test_df["generated_caption"] = generated_captions 
    return test_df


def get_bleu(test_df):
    unique_clips = test_df["youtube_id"].unique()
    targets,preds = [],[]
    for unique_clip in unique_clips:
        rows = test_df[test_df["youtube_id"] == unique_clip]
        target = rows["caption"].to_list()
        targets.append(target)
        
        pred = rows.iloc[0]["generated_caption"]
        preds.append(pred)
    
    print("-"*20,"BLEU Scores","-"*20)
    print("BLEU-1",bleu_scorers[0](preds,targets))
    print("BLEU-2",bleu_scorers[1](preds,targets))
    print("BLEU-3",bleu_scorers[2](preds,targets))
    print("BLEU-4",bleu_scorers[3](preds,targets))

In [4]:
ckpt3_result = join_result_with_test(test_df,"results/125m-25epochs-default-3_ckpt_best.json")
get_bleu(ckpt3_result)

-------------------- BLEU Scores --------------------
BLEU-1 tensor(0.1942)
BLEU-2 tensor(0.0191)
BLEU-3 tensor(0.)
BLEU-4 tensor(0.)


In [5]:
ckpt4_result = join_result_with_test(test_df,"results/125m-25epochs-default-4_ckpt.json")
get_bleu(ckpt4_result)

-------------------- BLEU Scores --------------------
BLEU-1 tensor(0.1774)
BLEU-2 tensor(0.0183)
BLEU-3 tensor(0.)
BLEU-4 tensor(0.)


In [6]:
ckpt18_result = join_result_with_test(test_df,"results/125m-25epochs-default-18_ckpt_best.json")
get_bleu(ckpt18_result)

-------------------- BLEU Scores --------------------
BLEU-1 tensor(0.1059)
BLEU-2 tensor(0.0102)
BLEU-3 tensor(0.)
BLEU-4 tensor(0.)


In [21]:
ckpt3_result.iloc[500]["generated_caption"]

'This is the sound of  the original, and the original is the original, and the original is the original'

In [24]:
ckpt4_result.iloc[500]["generated_caption"]

'This is the sound of  the original, and the original is the original, and the original is the original'

In [23]:
ckpt18_result.iloc[500]["generated_caption"]

'This is the sound of  the original, and the original is the original, and the original is the original'

In [13]:
ckpt3_result.head()

,audiocap_id,youtube_id,start_time,caption,generated_caption
0,10176,350OCezayrk,30,An engine is being started up then idles,This is the sound of the original.
1,102728,D9tinq3RMpU,30,A ship engine running as a crowd of people tal...,This is the sound of the city itself.
2,102729,sI7_ycEYzAY,30,A clock ticks low and loud and someone breathe...,This is the sound of the last of the last of ...
3,102730,WqXFAY4k79s,230,A girl speaks while rain pitter-patters follow...,This is the sound of the rain of the rain of ...
4,102733,yRoKi7rhSRo,20,A toilet flushing followed by a person speakin...,This is the sound of the last of the last of ...
